In [1]:
import tensorflow as tf
import numpy as np
from tftools import network as ntwrk

In [ ]:
VGG_MEAN = [103.939, 116.779, 123.68]

class VGGNtwrk(ntwrk.Network):
    def __init__(self,trainable=True):
        self.data = tf.placeholder(tf.float32, shape=[None, 224, 224, 3])
        red, green, blue = tf.split(3, 3, self.data)
        bgr = tf.concat(3, [
            blue - VGG_MEAN[0],
            green - VGG_MEAN[1],
            red - VGG_MEAN[2],
        ])
        self.layers = dict({'data':bgr})
        self.trainable = trainable
        self.setup()
        
    def setup(self):
        (self.feed('data')
             .conv(3, 3, 64, 1, 1, name='conv1_1')
             .conv(3, 3, 64, 1, 1, name='conv1_2')
             .max_pool(2, 2, 2, 2, padding='VALID', name='pool1')
             .conv(3, 3, 128, 1, 1, name='conv2_1')
             .conv(3, 3, 128, 1, 1, name='conv2_2')
             .max_pool(2, 2, 2, 2, padding='VALID', name='pool2')
             .conv(3, 3, 256, 1, 1, name='conv3_1')
             .conv(3, 3, 256, 1, 1, name='conv3_2')
             .conv(3, 3, 256, 1, 1, name='conv3_3')
             .max_pool(2, 2, 2, 2, padding='VALID', name='pool3')
             .conv(3, 3, 512, 1, 1, name='conv4_1')
             .conv(3, 3, 512, 1, 1, name='conv4_2')
             .conv(3, 3, 512, 1, 1, name='conv4_3')
             .max_pool(2, 2, 2, 2, padding='VALID', name='pool4')
             .conv(3, 3, 512, 1, 1, name='conv5_1')
             .conv(3, 3, 512, 1, 1, name='conv5_2')
             .conv(3, 3, 512, 1, 1, name='conv5_3')
             .max_pool(2,2,2,2, padding='VALID', name='pool5') 
             .fc(4096, name='fc6').fc(4096, name='fc7')
             .fc(1000,  name='fc8').softmax(name='prob'))

In [ ]:
sess = tf.Session()

vgg16 = VGGNtwrk()

vgg16.load('/data/image/models/VGG_imagenet.npy', sess)

In [6]:
VGG_MEAN = [103.939, 116.779, 123.68]

class ScorePredictor(ntwrk.Network):
    
    def normalize(self, im):
        red, green, blue = tf.split(3, 3, im)
        bgr = tf.concat(3, [
            blue - VGG_MEAN[0],
            green - VGG_MEAN[1],
            red - VGG_MEAN[2],
        ])
        return bgr
    
    def __init__(self,trainable=True):
        self.data_left = tf.placeholder(tf.float32, shape=[None, 224, 224, 3])
        self.data_right = tf.placeholder(tf.float32, shape=[None, 224, 224, 3])
        self.layers = dict({'data_left':self.normalize(self.data_left), 
                            'data_right':self.normalize(self.data_right)})
        self.trainable = trainable
        self.setup()
        
    def setup(self):
        (self.feed('data_left')
             .conv(3, 3, 64, 1, 1, name='conv1_1', trainable=False, reuse=False)
             .conv(3, 3, 64, 1, 1, name='conv1_2', trainable=False, reuse=False)
             .max_pool(2, 2, 2, 2, padding='VALID', name='pool1')
             .conv(3, 3, 128, 1, 1, name='conv2_1', trainable=False, reuse=False)
             .conv(3, 3, 128, 1, 1, name='conv2_2', trainable=False, reuse=False)
             .max_pool(2, 2, 2, 2, padding='VALID', name='pool2')
             .conv(3, 3, 256, 1, 1, name='conv3_1', trainable=False, reuse=False)
             .conv(3, 3, 256, 1, 1, name='conv3_2', trainable=False, reuse=False)
             .conv(3, 3, 256, 1, 1, name='conv3_3', trainable=False, reuse=False)
             .max_pool(2, 2, 2, 2, padding='VALID', name='pool3')
             .conv(3, 3, 512, 1, 1, name='conv4_1', reuse=False)
             .conv(3, 3, 512, 1, 1, name='conv4_2', reuse=False)
             .conv(3, 3, 512, 1, 1, name='conv4_3', reuse=False)
             .max_pool(2, 2, 2, 2, padding='VALID', name='pool4')
             .conv(3, 3, 512, 1, 1, name='conv5_1', reuse=False)
             .conv(3, 3, 512, 1, 1, name='conv5_2', reuse=False)
             .conv(3, 3, 512, 1, 1, name='conv5_3', reuse=False)
             .max_pool(2,2,2,2, padding='VALID', name='pool5') 
             .fc(512, name='fc6_common', reuse=False).fc(128, name='fc7_left'))
        
        (self.feed('data_right')
             .conv(3, 3, 64, 1, 1, name='conv1_1', trainable=False, reuse=True)
             .conv(3, 3, 64, 1, 1, name='conv1_2', trainable=False, reuse=True)
             .max_pool(2, 2, 2, 2, padding='VALID', name='pool1')
             .conv(3, 3, 128, 1, 1, name='conv2_1', trainable=False, reuse=True)
             .conv(3, 3, 128, 1, 1, name='conv2_2', trainable=False, reuse=True)
             .max_pool(2, 2, 2, 2, padding='VALID', name='pool2')
             .conv(3, 3, 256, 1, 1, name='conv3_1', trainable=False, reuse=True)
             .conv(3, 3, 256, 1, 1, name='conv3_2', trainable=False, reuse=True)
             .conv(3, 3, 256, 1, 1, name='conv3_3', trainable=False, reuse=True)
             .max_pool(2, 2, 2, 2, padding='VALID', name='pool3')
             .conv(3, 3, 512, 1, 1, name='conv4_1', reuse=True)
             .conv(3, 3, 512, 1, 1, name='conv4_2', reuse=True)
             .conv(3, 3, 512, 1, 1, name='conv4_3', reuse=True)
             .max_pool(2, 2, 2, 2, padding='VALID', name='pool4')
             .conv(3, 3, 512, 1, 1, name='conv5_1', reuse=True)
             .conv(3, 3, 512, 1, 1, name='conv5_2', reuse=True)
             .conv(3, 3, 512, 1, 1, name='conv5_3', reuse=True)
             .max_pool(2,2,2,2, padding='VALID', name='pool5') 
             .fc(512, name='fc6_common', reuse=True).fc(128, name='fc7_right'))
         
        (self.feed('fc7_left', 'fc7_right').concat(1).fc(32, name='fc8').fc(1, name='fc9', relu=False))
         
        


In [3]:
sess = tf.Session()

robnw = ScorePredictor()


In [5]:
robnw.load('/data/image/models/VGG_imagenet.npy', sess, ignore_missing=True)

In [ ]:
?tf.get_variable